In [2]:
from clickhouse_connect import get_client
from truedata.analytics import TD_analytics
import pandas as pd
import numpy as np
import logging
import pytz
from datetime import datetime, timedelta, time as dtime, timezone, date
import warnings
warnings.filterwarnings('ignore')

IST = pytz.timezone("Asia/Kolkata")
IST = timezone(timedelta(hours=5, minutes=30))

today = datetime.now(IST).date()

# Config

In [3]:
# Clickhouse credentials
host = "localhost"
port = 8123
username = "ingest_w"
password = "ingest_secret"
database = "market"

# TrueData Analytics credentials
td_username = "True9030"
td_password = "vineet@9030"
td_log_level = logging.WARNING

# Connect to ClickHouse | TrueData Analytics

In [4]:
def get_clickhouse_conn(host,port,username,password,database):
    try:
        conn = get_client(
            host=host,
            port=port,
            username=username,
            password=password,
            database=database
        )
        conn.query("SELECT 1")
        print("✅ Connected to ClickHouse database!")
        print(f"📊 Database: {conn.database}")
        print(f"🕐 Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        return conn
    except Exception as e:
        return f"Exception in get_clickhouse_conn as : {e}"

In [5]:
conn = get_clickhouse_conn(host,port,username,password,database)
obj_td_analytics = TD_analytics(td_username, td_password, log_level=td_log_level)

✅ Connected to ClickHouse database!
📊 Database: market
🕐 Current time: 2025-11-06 16:10:05


(2025-11-06 16:10:05,849) WARNING :: Connected successfully to TrueData Anaytical Data Service...  (PID:15160 Thread:9400)


# Functions

In [6]:
def get_strike_step(symbol):
    if "BANKNIFTY" in symbol:
        return 100
    elif "NIFTY" in symbol:
        return 50
    elif "FINNIFTY" in symbol:
        return 50
    return 1


def _fetch_iv_by_expiry_today(conn,SYMBOL):

    today = date.today().isoformat()

    #! 1. Get all rows for today and SYMBOL
    q = '''
        SELECT snap_minute_ist, expiry, strike, side, iv
        FROM option_chain_1m
        WHERE underlying = %(symbol)s
          AND toDate(snap_minute_ist) = %(today)s
        ORDER BY expiry, snap_minute_ist, strike
    '''
    rows = list(conn.query(q, parameters={"symbol": SYMBOL, "today": today}).named_results())
    if not rows:
        return pd.DataFrame(columns=["time","expiry_ts","expiry_str","tenor_tag","atm_strike","ce_iv","pe_iv","iv_avg"])
    df = pd.DataFrame(rows)
    df['expiry_str'] = df['expiry'].astype(str)
    df['expiry_ts'] = pd.to_datetime(df['expiry']).astype(int) // 10**9

    #! 2. Find the two nearest expiries (sorted ascending)
    expiry_dates = sorted(df['expiry'].unique())
    if len(expiry_dates) < 1:
        return pd.DataFrame(columns=["time","expiry_ts","expiry_str","tenor_tag","atm_strike","ce_iv","pe_iv","iv_avg"])
    near_expiry = expiry_dates[0]
    near_dt = pd.to_datetime(near_expiry)
    # Find all expiries in the next month after near expiry's month
    far_month_expiries = [e for e in expiry_dates if (pd.to_datetime(e).year > near_dt.year) or (pd.to_datetime(e).year == near_dt.year and pd.to_datetime(e).month == near_dt.month + 1)]
    if far_month_expiries:
        far_month = pd.to_datetime(far_month_expiries[0]).month
        far_year = pd.to_datetime(far_month_expiries[0]).year
        # Get all expiries in that month
        far_expiries_in_month = [e for e in expiry_dates if pd.to_datetime(e).year == far_year and pd.to_datetime(e).month == far_month]
        far_expiry = max(far_expiries_in_month)
        expiry_map = {near_expiry: 'near', far_expiry: 'far'}
    else:
        expiry_map = {near_expiry: 'near'}

    #! 3. Get spot LTP and strike step for ATM calculation
    ul_ltp = float(obj_td_analytics.get_spot_ltp(SYMBOL).LTP.iloc[0])
    strike_step = get_strike_step(SYMBOL)

    #! 4. For each expiry, for each minute, get ATM strike and IVs
    out_rows = []
    for expiry, tenor_tag in expiry_map.items():
        df_exp = df[df['expiry'] == expiry].copy()
        # For each minute
        for minute, group in df_exp.groupby(df_exp['snap_minute_ist']):
            # Find ATM strike for this minute
            strikes = group['strike'].unique()
            if len(strikes) == 0:
                continue
            # ATM = closest to ul_ltp, snapped to strike step
            atm_strike = min(strikes, key=lambda s: abs(float(s) - ul_ltp))
            # Get CE/PE IVs for ATM
            ce_iv = group[(group['strike'] == atm_strike) & (group['side'] == 'C')]['iv']
            pe_iv = group[(group['strike'] == atm_strike) & (group['side'] == 'P')]['iv']
            ce_iv = float(ce_iv.iloc[0]) if not ce_iv.empty else None
            pe_iv = float(pe_iv.iloc[0]) if not pe_iv.empty else None
            iv_avg = 0.5 * (ce_iv + pe_iv) if ce_iv is not None and pe_iv is not None else None
            out_rows.append({
                'time': pd.to_datetime(minute).strftime('%H:%M'),
                'expiry_ts': int(pd.to_datetime(expiry).timestamp()),
                'expiry_str': str(expiry),
                'tenor_tag': tenor_tag,
                'atm_strike': atm_strike,
                'ce_iv': ce_iv,
                'pe_iv': pe_iv,
                'iv_avg': iv_avg
            })
    out_df = pd.DataFrame(out_rows)
    out_df[['ce_iv', 'pe_iv', 'iv_avg']] = out_df[['ce_iv', 'pe_iv', 'iv_avg']] * 100
    return out_df.sort_values(['time','tenor_tag']).reset_index(drop=True)


def _compute_vix30(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    today = datetime.now(IST).date()
    df = df.copy()
    df["t_dt"] = pd.to_datetime(df["time"], format="%H:%M").map(lambda t: datetime.combine(today, t.time(), tzinfo=IST))
    df["expiry_dt"] = pd.to_datetime(df["expiry_ts"], unit="s", utc=True).dt.tz_convert(IST)

    out = []
    T30 = 30.0/365.0
    for t, blk in df.groupby("t_dt", sort=True):
        blk = blk.copy()
        blk["days_left"] = (blk["expiry_dt"] - t).dt.total_seconds()/86400.0
        near = blk[blk["days_left"].between(0, 30, inclusive="left")].sort_values("days_left").head(1)
        far  = blk[blk["days_left"] >= 30].sort_values("days_left").head(1)

        vix = None
        if not near.empty and not far.empty:
            iv1 = float(near["iv_avg"].iloc[0]) / 100.0
            iv2 = float(far["iv_avg"].iloc[0])  / 100.0
            T1  = float(near["days_left"].iloc[0]) / 365.0
            T2  = float(far["days_left"].iloc[0])  / 365.0
            V1, V2 = iv1*iv1*T1, iv2*iv2*T2
            w1 = (T2 - T30) / (T2 - T1) if T2 != T1 else 1.0
            V30 = w1*V1 + (1.0 - w1)*V2
            vix = 100.0*np.sqrt(V30 / T30)
        else:
            use = near if not near.empty else (far if not far.empty else None)
            if use is not None and not use.empty:
                iv = float(use["iv_avg"].iloc[0]) / 100.0
                T  = float(use["days_left"].iloc[0]) / 365.0
                if T > 0:
                    vix = 100.0 * iv * np.sqrt(T / T30)

        if vix is not None:
            out.append({"t_dt": t, "time": t.strftime("%H:%M"), "vix30": round(float(vix), 2)})

    print(
        f"{t:%H:%M} near_iv%={near['iv_avg'].iloc[0]:.2f} "
        f"near_days={near['days_left'].iloc[0]:.1f}  "
        f"far_iv%={(far['iv_avg'].iloc[0] if not far.empty else float('nan')):.2f} "
        f"far_days={(far['days_left'].iloc[0] if not far.empty else float('nan')):.1f}"
    )

    return pd.DataFrame(out).sort_values("t_dt")


def _resample_vixdf(vixdf: pd.DataFrame, interval_min: int) -> pd.DataFrame:
    if vixdf.empty or int(interval_min) <= 1:
        return vixdf
    d = vixdf.copy()
    # ensure we have a datetime index
    if "t_dt" not in d.columns:
        today = datetime.now(IST).date()
        d["t_dt"] = pd.to_datetime(d["time"], format="%H:%M").map(
            lambda t: datetime.combine(today, t.time(), tzinfo=IST)
        )
    d = d.set_index("t_dt").sort_index()
    # last value per bucket
    out = d.resample(f"{int(interval_min)}min")[["vix30"]].last().dropna(how="all").reset_index()
    out["time"] = out["t_dt"].dt.strftime("%H:%M")
    return out[["t_dt", "time", "vix30"]]


def fetch_vix_for_indices(conn, indices, interval_min):
    all_vixdfs = []  # to store vix dataframes for each index

    for index in indices:
        rows = _fetch_iv_by_expiry_today(conn, index)
        if not rows.empty:
            vixdf = _compute_vix30(rows)
            vixdf = _resample_vixdf(vixdf, interval_min)

            # Add instrument column
            vixdf['instrument'] = index

            # Rename t_dt to date (optional — or keep as is)
            vixdf['date'] = pd.to_datetime(vixdf['t_dt']).dt.date
            vixdf['time'] = pd.to_datetime(vixdf['t_dt']).dt.strftime('%H:%M')

            # Keep only required columns
            vixdf = vixdf[['date', 'time', 'instrument', 'vix30']]

            all_vixdfs.append(vixdf)

    # Combine all instruments into one DataFrame
    if all_vixdfs:
        final_df = pd.concat(all_vixdfs, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame(columns=['date', 'time', 'instrument', 'vix30'])

# Vix

In [7]:
index_list = ["NIFTY", "BANKNIFTY", "MIDCPNIFTY", "FINNIFTY", "SENSEX", "BANKEX"]
interval_min_list = [1, 3, 5, 15]

For multiple indices

In [66]:
indices = ["NIFTY", "BANKNIFTY"]
interval_min = 1

final_vixdf = fetch_vix_for_indices(conn, indices, interval_min)
final_vixdf

13:48 near_iv%=12.27 near_days=0.7  far_iv%=11.65 far_days=56.7
13:48 near_iv%=12.56 near_days=21.7  far_iv%=12.04 far_days=56.7


,date,time,instrument,vix30
0,2025-11-03,10:30,NIFTY,11.71
1,2025-11-03,10:31,NIFTY,11.75
2,2025-11-03,10:32,NIFTY,11.72
3,2025-11-03,10:33,NIFTY,11.73
4,2025-11-03,10:34,NIFTY,11.72
...,...,...,...,...
393,2025-11-03,13:44,BANKNIFTY,12.32
394,2025-11-03,13:45,BANKNIFTY,12.32
395,2025-11-03,13:46,BANKNIFTY,12.35
396,2025-11-03,13:47,BANKNIFTY,12.34


For single index

In [16]:
rows

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg


In [ ]:
index = "DIXON"
interval_min = 1

rows = _fetch_iv_by_expiry_today(conn, index)

if not rows.empty:
    vixdf = _compute_vix30(rows)
    vixdf = _resample_vixdf(vixdf, interval_min)
else:
    vixdf = pd.DataFrame(columns=["date", "time", "instrument", "vix30"])
    
vixdf

,date,time,instrument,vix30


# Rough

In [ ]:
index = "NIFTY"
interval_min = 1

rows = _fetch_iv_by_expiry_today(conn, index)
if not rows.empty:
    vixdf = _compute_vix30(rows)
vixdf = _resample_vixdf(vixdf, interval_min)

13:05 near_iv%=0.12 near_days=27.7  far_iv%=nan far_days=nan


,t_dt,time,vix30
0,2025-10-28 10:25:00+05:30,10:25,0.11
1,2025-10-28 10:26:00+05:30,10:26,0.11
2,2025-10-28 10:27:00+05:30,10:27,0.11
3,2025-10-28 10:28:00+05:30,10:28,0.11
4,2025-10-28 10:29:00+05:30,10:29,0.11
...,...,...,...
156,2025-10-28 13:01:00+05:30,13:01,0.11
157,2025-10-28 13:02:00+05:30,13:02,0.11
158,2025-10-28 13:03:00+05:30,13:03,0.11
159,2025-10-28 13:04:00+05:30,13:04,0.11


In [23]:
rows

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg
0,10:25,1764028800,2025-11-25,far,25950.00,0.13229,0.10548,0.118885
1,10:25,1761609600,2025-10-28,near,25950.00,0.17169,0.14317,0.157430
2,10:26,1764028800,2025-11-25,far,25950.00,0.13123,0.10571,0.118470
3,10:26,1761609600,2025-10-28,near,25950.00,0.17041,0.14652,0.158465
4,10:27,1764028800,2025-11-25,far,25950.00,0.13152,0.10593,0.118725
...,...,...,...,...,...,...,...,...
177,11:53,1761609600,2025-10-28,near,25950.00,0.13839,0.11379,0.126090
178,11:54,1764028800,2025-11-25,far,25950.00,0.13074,0.10303,0.116885
179,11:54,1761609600,2025-10-28,near,25950.00,0.14057,0.11234,0.126455
180,11:55,1764028800,2025-11-25,far,25950.00,0.12983,0.10357,0.116700


In [27]:
today = datetime.now(IST).date()
df = rows.copy()
df["t_dt"] = pd.to_datetime(rows["time"], format="%H:%M").map(lambda t: datetime.combine(today, t.time(), tzinfo=IST))
df["expiry_dt"] = pd.to_datetime(rows["expiry_ts"], unit="s", utc=True).dt.tz_convert(IST)

In [28]:
df

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg,t_dt,expiry_dt
0,10:25,1764028800,2025-11-25,far,25950.00,0.13229,0.10548,0.118885,2025-10-28 10:25:00+05:30,2025-11-25 05:30:00+05:30
1,10:25,1761609600,2025-10-28,near,25950.00,0.17169,0.14317,0.157430,2025-10-28 10:25:00+05:30,2025-10-28 05:30:00+05:30
2,10:26,1764028800,2025-11-25,far,25950.00,0.13123,0.10571,0.118470,2025-10-28 10:26:00+05:30,2025-11-25 05:30:00+05:30
3,10:26,1761609600,2025-10-28,near,25950.00,0.17041,0.14652,0.158465,2025-10-28 10:26:00+05:30,2025-10-28 05:30:00+05:30
4,10:27,1764028800,2025-11-25,far,25950.00,0.13152,0.10593,0.118725,2025-10-28 10:27:00+05:30,2025-11-25 05:30:00+05:30
...,...,...,...,...,...,...,...,...,...,...
177,11:53,1761609600,2025-10-28,near,25950.00,0.13839,0.11379,0.126090,2025-10-28 11:53:00+05:30,2025-10-28 05:30:00+05:30
178,11:54,1764028800,2025-11-25,far,25950.00,0.13074,0.10303,0.116885,2025-10-28 11:54:00+05:30,2025-11-25 05:30:00+05:30
179,11:54,1761609600,2025-10-28,near,25950.00,0.14057,0.11234,0.126455,2025-10-28 11:54:00+05:30,2025-10-28 05:30:00+05:30
180,11:55,1764028800,2025-11-25,far,25950.00,0.12983,0.10357,0.116700,2025-10-28 11:55:00+05:30,2025-11-25 05:30:00+05:30


In [35]:
out = []
T30 = 30.0/365.0
for t, blk in df.groupby("t_dt", sort=True):
    blk = blk.copy()
    blk["days_left"] = (blk["expiry_dt"] - t).dt.total_seconds()/86400.0
    # near = blk[blk["days_left"].between(0, 30, inclusive="left")].sort_values("days_left").head(1)
    near = blk[blk["days_left"] >= -0.5].sort_values("days_left").head(1)
    far  = blk[blk["days_left"] >= 30].sort_values("days_left").head(1)
    vix = None
    if not near.empty and not far.empty:
        iv1 = float(near["iv_avg"].iloc[0]) / 100.0
        iv2 = float(far["iv_avg"].iloc[0])  / 100.0
        T1  = float(near["days_left"].iloc[0]) / 365.0
        T2  = float(far["days_left"].iloc[0])  / 365.0
        V1, V2 = iv1*iv1*T1, iv2*iv2*T2
        w1 = (T2 - T30) / (T2 - T1) if T2 != T1 else 1.0
        V30 = w1*V1 + (1.0 - w1)*V2
        vix = 100.0*np.sqrt(V30 / T30)
        print(f"vix: {vix}")
    else:
        use = near if not near.empty else (far if not far.empty else None)
        if use is not None and not use.empty:
            iv = float(use["iv_avg"].iloc[0]) / 100.0
            T  = float(use["days_left"].iloc[0]) / 365.0
            if T > 0:
                vix = 100.0 * iv * np.sqrt(T / T30)
    if vix is not None:
        out.append({"t_dt": t, "time": t.strftime("%H:%M"), "vix30": round(float(vix), 2)})
print(
    f"{t:%H:%M} near_iv%={near['iv_avg'].iloc[0]:.2f} "
    f"near_days={near['days_left'].iloc[0]:.1f}  "
    f"far_iv%={(far['iv_avg'].iloc[0] if not far.empty else float('nan')):.2f} "
    f"far_days={(far['days_left'].iloc[0] if not far.empty else float('nan')):.1f}"
)

11:55 near_iv%=0.13 near_days=-0.3  far_iv%=nan far_days=nan


In [38]:
blk

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg,t_dt,expiry_dt,days_left
180,11:55,1764028800,2025-11-25,far,25950.00,0.12983,0.10357,0.11670,2025-10-28 11:55:00+05:30,2025-11-25 05:30:00+05:30,27.732639
181,11:55,1761609600,2025-10-28,near,25950.00,0.13525,0.11547,0.12536,2025-10-28 11:55:00+05:30,2025-10-28 05:30:00+05:30,-0.267361


In [41]:
near = blk[blk["days_left"] >= -0.5].sort_values("days_left").head(1)
near

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg,t_dt,expiry_dt,days_left
181,11:55,1761609600,2025-10-28,near,25950.00,0.13525,0.11547,0.12536,2025-10-28 11:55:00+05:30,2025-10-28 05:30:00+05:30,-0.267361


In [ ]:
far  = blk[blk["days_left"] >= 30].sort_values("days_left").head(1)

In [39]:
near

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg,t_dt,expiry_dt,days_left
181,11:55,1761609600,2025-10-28,near,25950.00,0.13525,0.11547,0.12536,2025-10-28 11:55:00+05:30,2025-10-28 05:30:00+05:30,-0.267361


In [40]:
far

,time,expiry_ts,expiry_str,tenor_tag,atm_strike,ce_iv,pe_iv,iv_avg,t_dt,expiry_dt,days_left


In [31]:
vix

In [ ]:
return pd.DataFrame(out).sort_values("t_dt")